## Package Install

In [12]:
!pip install openai==0.28
!pip install paddlepaddle paddleocr
!pip install pdfminer.six
!pip install pypdf2
!pip install pdf2image
!apt-get install -y poppler-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.12.0
    Uninstalling openai-1.12.0:
      Successfully uninstalled openai-1.12.0


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


## 事前準備

In [2]:
# OpenAI APIキーの設定
openai_key = "" #@param {type:"string"}

In [3]:
# 書類ジャンル (これを入力しない場合はジャンル推定処理が走る)
# 例: 請求書, 納品書, レシート, etc...
genre = "" #@param {type:"string"}

In [5]:
from google.colab import files
uploaded = files.upload()
file_path = next(iter(uploaded.keys()))

Saving 手書き文字.png to 手書き文字 (1).png


## Functions

In [6]:
from typing import List, Dict, Union
from paddleocr import PaddleOCR
from pdf2image import convert_from_path
from PIL import Image
import openai
import re
import json
import os
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTTextBox, LTTextLine
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from PyPDF2 import PdfFileReader
from pdfminer.pdffont import PDFType1Font, PDFTrueTypeFont


def estimate_genre(ocr_text: str, openai_key: str) -> str:
    """
    OCRで得たテキスト情報から書類ジャンルを推定する

    Parameters
    ----------
    ocr_text: str
        OCRで抽出したテキスト情報
    openai_key: str
        OpenAI API Key

    Returns
    -------
    genre: str
        書類ジャンル
    """

    functions = [
        {
            "name": "genre_estimate",
            "description": "これはある書面の画像をOCRにかけたものから、書類のジャンルを推定するための処理です。OCRで抽出されたテキストは以下の形式に従います: (x座標, y座標): {OCRで抽出されたテキスト}",
            "parameters": {
                "type": "object",
                "properties": {
                    "genre_name": {
                        "type": "string",
                        "description": "ジャンル名 (例: 請求書, 領収書, 運転免許証, はがき, レポート, etc...)"
                    }
                },
                "required": ["genre_name"]
            }
        }
    ]
    result = process_with_gpt(ocr_text, functions, openai_key)
    return result['genre_name']

def get_functions_from_gpt(genre: str, openai_key: str) -> List[Dict[str, Union[str, int]]]:
    """
    指定した書類ジャンルのFunction calling用Functionsを作成する

    Parameters
    ----------
    genre: str
        書類ジャンル
    openai_key: str
        OpenAI API Key

    Returns
    -------
    functions: List[Dict[str, Union[str, int]]]
        Function calling用Functions
    """
    function_prompt = '''情報抽出システムがあります。
規定のフォーマットを参考に、これを「xxxx」に対応したものに書き換え、その結果のみ出力してください。

## 規定のフォーマット

```
functions = [
    {
        "name": "invoice_information_extraction",
        "description": "これは請求書のpdfをOCRにかけたものから情報を抽出するための処理です。OCRで抽出されたテキストは以下の形式に従います: (x座標, y座標): {OCRで抽出されたテキスト} また、請求書は以下の配置ルールがあります。(1) 銀行名、支店名、口座種類、口座番号は座標的に近い位置にある (2) 住所、請求元会社名、名前、電話番号、メールアドレスは座標的に近い位置にある (3) 請求元会社名は請求先とは異なり、無い場合がある (4) 郵便番号、住所は座標的に近い位置にある (5) 請求先は「御中」や「様」などの左に書かれる\n座標を考慮しつつ情報を抽出してください。",
        "parameters": {
            "type": "object",
            "properties": {
                "invoice_date": {
                    "type": "string",
                    "description": "請求日"
                },
                "invoice_number": {
                    "type": "string",
                    "description": "請求番号"
                },
                "invoice_to": {
                    "type": "string",
                    "description": "請求先 (御中とか様は除外する)"
                },
                "billing_amount": {
                    "type": "number",
                    "description": "請求する金額"
                },
                "invoice_items": {
                    "type": "array",
                    "description": "請求品目",
                    "items": {
                        "type": "object",
                        "properties": {
                            "item_name": {
                                "type": "string",
                                "description": "請求品目名"
                            },
                            "item_count": {
                                "type": "number",
                                "description": "品目数量"
                            },
                            "item_unit_name": {
                                "type": "string",
                                "description": "品目単位名"
                            },
                            "item_unit_price": {
                                "type": "number",
                                "description": "品目単価"
                            },
                            "item_price": {
                                "type": "number",
                                "description": "品目金額"
                            }
                        }
                    }
                },
                "bank_name": {
                    "type": "string",
                    "description": "銀行名"
                },
                "branch_name": {
                    "type": "string",
                    "description": "支店名"
                },
                "account_type": {
                    "type": "string",
                    "enum": ["普通", "定期", "当座"],
                    "description": "口座種類"
                },
                "account_number": {
                    "type": "string",
                    "description": "口座番号"
                },
                "zipcode": {
                    "type": "string",
                    "description": "郵便番号"
                },
                "address": {
                    "type": "string",
                    "description": "住所"
                },
                "company_name": {
                    "type": "string",
                    "description": "請求元会社名"
                },
                "name": {
                    "type": "string",
                    "description": "名前"
                },
                "phone_number": {
                    "type": "string",
                    "description": "電話番号"
                },
                "email_address": {
                    "type": "string",
                    "description": "メールアドレス"
                },
                "remarks": {
                    "type": "string",
                    "description": "備考"
                }
            },
            "required": ["invoice_date"]
        }
    }
]
```'''.replace('xxxx', genre)
    function_messages = [{"role": "user", "content": function_prompt}]

    openai.api_key = openai_key
    function_response = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=function_messages,
        temperature=0,
        max_tokens=1024,
    )
    function_text = function_response['choices'][0]['message']['content']

    match = re.search(r'functions = (\[.*\])', function_text, re.DOTALL)
    if match:
        extracted_json_string = match.group(1)
    else:
        raise ValueError("No valid JSON object found")

    # json形式の文字列をPythonのリストに変換
    functions = json.loads(extracted_json_string.replace("\n", ""))
    return functions

def extract_text_with_ocr(file_path: str, ocr: PaddleOCR) -> str:
    """
    指定したファイルからOCRを使用してテキストを抽出する関数

    Parameters
    ----------
    file_path: str
        テキストを抽出するファイルのパス

    Returns
    -------
    ocr_text: str
        OCRで抽出したテキスト情報
    """
    # ファイル拡張子を取得します
    extension = os.path.splitext(file_path)[1]

    results = []

    if extension.lower() in ['.pdf']:
        images = convert_from_path(file_path)
        for i, image in enumerate(images, start=1):
            print(f"Processing page {i}")
            image_path = f'{i}.png'
            image.save(image_path, 'PNG')
            result = ocr.ocr(image_path)
            results.append(result)

    elif extension.lower() in ['.jpg', '.jpeg', '.png']:
        print(f"Processing image {file_path}")
        result = ocr.ocr(file_path)
        results.append(result)

    else:
        print(f"Unsupported file type {extension}")

    ocr_text = ""
    for r in results[0][0]:
        ocr_text += f"({r[0][0][0]}, {r[0][0][1]}): {r[1][0]}\n"

    return ocr_text

def extract_text_with_pdfminer(file_path: str) -> str:
    """
    指定したPDFファイルからpdfminerを使用してテキストを抽出する関数

    Parameters
    ----------
    file_path: str
        テキストを抽出するPDFファイルのパス

    Returns
    -------
    ocr_text: str
        pdfminerで抽出したテキスト情報
    """
    ocr_text = ""

    with open(file_path, 'rb') as file:
        parser = PDFParser(file)
        doc = PDFDocument(parser)

        rsrcmgr = PDFResourceManager()
        laparams = LAParams()
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)

        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)
            layout = device.get_result()
            for element in layout:
                if isinstance(element, LTTextBox) or isinstance(element, LTTextLine):
                    for text_line in element:
                        if isinstance(text_line, LTTextLine):
                            x, y, _, _ = text_line.bbox
                            x = round(x, 1)  # x座標を小数点第1位までに丸める
                            y = round(y, 1)  # y座標を小数点第1位までに丸める
                            text = f"({x}, {y}): {text_line.get_text()}"
                            ocr_text += text

    return ocr_text

def process_with_gpt(ocr_text: str, functions: List[Dict[str, Union[str, int]]], openai_key: str) -> Dict[str, Union[str, int]]:
    """
    Function calling

    Parameters
    ----------
    ocr_text: str
        OCRで取得したテキスト情報
    functions: List[Dict[str, Union[str, int]]]
        Function calling用Function
    openai_key: str
        OpenAI API Key

    Returns
    -------
    result: Dict[str, Union[str, int]]
        GPT-3.5で処理した結果
    """
    messages = [{"role": "user", "content": ocr_text}]

    openai.api_key = openai_key
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        temperature=0,
        max_tokens=1024,
        function_call={"name": functions[0]['name']}
    )

    result = response["choices"][0]["message"]["function_call"]["arguments"]
    return json.loads(result)

def check_pdf_and_font(filepath: str) -> bool:
    """
    指定したファイルがPDFかつフォント埋め込みかチェックする関数

    Parameters
    ----------
    file_path: str
        テキストを抽出するPDFファイルのパス

    Returns
    -------
    bool
        pdfかつフォント埋め込みの場合True, それ以外の場合False
    """
    if os.path.exists(filepath) and filepath.endswith('.pdf'):
        try:
            reader = PdfFileReader(filepath)

            with open(filepath, 'rb') as f:
                parser = PDFParser(f)
                doc = PDFDocument(parser)

                fonts = set()
                for obj in doc.get_objects():
                    if isinstance(obj, (PDFType1Font, PDFTrueTypeFont)):
                        fonts.add(obj)

                if fonts:
                    return True
        except:
            pass

    return False

## 処理

In [21]:
# OCRモデルの初期化
ocr = PaddleOCR(lang='japan')

[2024/02/24 06:09:11] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/ml/Multilingual_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/japan/japan_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_

In [22]:
# OCR処理 (ジャンル推定にも使う)
ocr_text = extract_text_with_ocr(file_path, ocr)

Processing image 手書き文字 (1).png
[2024/02/24 06:09:12] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/02/24 06:09:13] ppocr DEBUG: dt_boxes num : 50, elapsed : 0.20905542373657227
[2024/02/24 06:09:16] ppocr DEBUG: rec_res num  : 50, elapsed : 3.5058517456054688


In [24]:
if genre == "":
    genre = estimate_genre(ocr_text, openai_key)

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
print(genre)

In [ ]:
# functionsを作成
functions = get_functions_from_gpt(genre, openai_key)

In [ ]:
print(functions)

In [ ]:
result = process_with_gpt(ocr_text, functions, openai_key)

In [ ]:
print(ocr_text)

In [ ]:
result

In [ ]:
# PDFにフォントが埋め込まれていたら以下も実行
is_pdf_with_embedded_fonts = check_pdf_and_font(file_path)
if is_pdf_with_embedded_fonts:
    miner_text = extract_text_with_pdfminer(file_path)
    result2 = process_with_gpt(miner_text, functions, openai_key)

In [ ]:
if is_pdf_with_embedded_fonts:
    print(miner_text)

In [ ]:
if is_pdf_with_embedded_fonts:
    result2